In [20]:
import pandas as pd
import nltk
import requests
import bs4
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import re

from pprint import pprint

# Issues in code
1. Unable to find medicines with - or space dependency from mims website. Need to resolve this issue
2. Unable to find some medicines in mims website, need another website to scrape

In [37]:
def clean_text(text):
    lst = []
    pattern = "[\d\.\s]+(mg[//]?|ml)+"
    clean_string = re.sub(pattern, ' ', text )
    
    pattern = "\([a-z]+\)"
    clean_string = re.sub(pattern, ' ', clean_string )

    pattern = ":\s*[a-z\,\-\s]+"
    final_string = re.findall(pattern, clean_string )
    
    
    for drug in final_string:
        lst.append(drug.split(":")[1])
        
    return lst

In [38]:
def clean_text_without_colon(text):
    
#     pattern = "[\r\n]"
#     clean_string = re.sub(pattern, '', text )
    
    pattern = "[a-z,\s]+"
    clean_string = re.findall(pattern, text)
    
    # Assuming all text in the first string only
    lst = []
    lst.append(clean_string[0])
    return lst

In [39]:
def remove_common_drugs(drugs_list):
    updated_drug_list = []
    for drug in drugs_list:
        drug = drug.strip()

        if len(updated_drug_list) == 0:
            updated_drug_list.append(drug)
        else:
            if drug not in updated_drug_list:
                updated_drug_list.append(drug)
    return updated_drug_list

In [8]:
def correct_dict(drug_dict):
    for key, value in drug_dict.items():
        lst = []
        if len(value) > 1:
            for item in value:
                if ',' in item:
                    if len(item) > 1:
                        for sub_item in item.split(','):
                            lst.append(sub_item)
                        drug_dict[key] = lst
        else:
            for item in value:
                if "," in item:
                    for sub_item in item.split(","):
                        lst.append(sub_item)
                    drug_dict[key] = lst
    return drug_dict


In [46]:
# med_names = ["pantop", "levera", "cardace", "dytor", "benfomet plus", "aldactone", "nexito", "aristozyme", "roliflo-od", "atorva"]


med_names = ["Valsac", "roseday gold", "asthalin", "silverex", "lysoflam", "surfaz-sn", "volini"]

In [47]:
all_drug_names_dict = {}
for med_name in med_names:
    url = 'https://www.mims.com/india/drug/info/' + med_name
    request_result=requests.get( url )
    soup = bs4.BeautifulSoup(request_result.text, "html.parser")
    
    try:
        medicine_raw_text = soup.find("div", "monograph-section-content").text
        
        #Removing white spaces and tabs
        medicine_raw_text = re.sub("[\n\r]", "", medicine_raw_text)
        
        # Making all text lowercase
        medicine_raw_text = medicine_raw_text.lower()
        
        # Resolving text issues for cases with ":"
        if ":" in medicine_raw_text:
            drug_list = clean_text(medicine_raw_text)
        else:
            drug_list = clean_text_without_colon(medicine_raw_text)
            
        all_drug_names_dict[med_name] = drug_list
    except Exception as e:
        print(e)
        print("No drug found for {}".format(med_name))

'NoneType' object has no attribute 'text'
No drug found for Valsac


In [48]:
all_drug_names_dict

{'roseday gold': [' aspirin , rosuvastatin , clopidogrel '],
 'asthalin': [' salbutamol ', ' salbutamol ', ' salbutamol ', ' salbutamol'],
 'silverex': [' silver sulphadiazine ', ' silver sulphadiazine '],
 'lysoflam': [' paracetamol , trypsin '],
 'surfaz-sn': [' clotrimazole '],
 'volini': ['diclofenac sodium']}

In [49]:
correct_dict(all_drug_names_dict)

{'roseday gold': [' aspirin ', ' rosuvastatin ', ' clopidogrel '],
 'asthalin': [' salbutamol ', ' salbutamol ', ' salbutamol ', ' salbutamol'],
 'silverex': [' silver sulphadiazine ', ' silver sulphadiazine '],
 'lysoflam': [' paracetamol ', ' trypsin '],
 'surfaz-sn': [' clotrimazole '],
 'volini': ['diclofenac sodium']}

In [50]:
for med_name, drug_list in all_drug_names_dict.items():
    
    clean_drug_list = remove_common_drugs(drug_list)
    all_drug_names_dict[med_name] = clean_drug_list

In [51]:
for key, value in all_drug_names_dict.items():
    print("{}    : {}".format(key, value))

roseday gold    : ['aspirin', 'rosuvastatin', 'clopidogrel']
asthalin    : ['salbutamol']
silverex    : ['silver sulphadiazine']
lysoflam    : ['paracetamol', 'trypsin']
surfaz-sn    : ['clotrimazole']
volini    : ['diclofenac sodium']
